# Create CaCO3 crystals with different surface terminations

## Set up

Import dependencies

In [1]:
import MDAnalysis as mda
from MDAnalysis import transformations
import numpy as np
from openbabel import openbabel as ob
import os
from pymatgen.core.structure import Structure
from pymatgen.core.surface import SlabGenerator
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.io.babel import BabelMolAdaptor
import warnings

/home/aglisman/miniconda3/envs/pymatgen/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/aglisman/miniconda3/envs/pymatgen/lib/python3.9/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


Helper functions

In [2]:
def conv_hex_to_cubic_idx(hex_idx):
    """
    Converts a hexagonal index to a cubic index.
    """
    assert(len(hex_idx) == 4)
    return (hex_idx[0], hex_idx[1], hex_idx[3])

## Data source and preparation

In [3]:
# location of calcite cif file
crystals = ["calcite_297K", "aragonite", "vaterite"]
crystal_dims = {
    "calcite_297K": (0.50, 0.43),
    "aragonite": (0.50, 0.81),
    "vaterite": (0.50, 0.81),
}

# which crystal to use
crystal = crystals[0]
# whether to create a supercell by replicating the unit cell
supercell = True
# box length in nanometers
cubic_box_length_nm = 12
# slab thickness in angstroms
slab_thickness = 9
# miller indices of the surface
miller = conv_hex_to_cubic_idx((0, 0, 0, 1))


calcite_file = f"./../american-mineralogist-crystal-structure-database/{crystal}/AMS_DATA.cif"


# number of cells in the slab (z-axis is perpendicular to the surface)
if supercell:
    supercell_size = (
        np.round(cubic_box_length_nm / crystal_dims[crystal][0]),
        np.round(cubic_box_length_nm / crystal_dims[crystal][1]),
        1
    )
else:
    supercell_size = (1, 1, 1)


# vacuum thickness in angstroms
vacuum_thickness = (cubic_box_length_nm * 10) - slab_thickness


### Conventional unit cell

In [4]:
# load calcite crystal structure
calcite = Structure.from_file(calcite_file, primitive=False)
calcite.add_oxidation_state_by_element({"Ca": 2, "C": 4, "O": -2})
sga = SpacegroupAnalyzer(calcite)
calcite_conv = sga.get_conventional_standard_structure()

# print surface slab data
input = calcite_conv
print("Conventional calcite cell")
print(f"Calcite space group is {sga.get_space_group_symbol()}")
print(f"Is calcite lattice hexagonal? {input.lattice.is_hexagonal()}")
print()

info = str(input).split("\n")
for i in range(5):
    print(info[i])

Conventional calcite cell
Calcite space group is R-3c
Is calcite lattice hexagonal? True

Full Formula (Ca6 C6 O18)
Reduced Formula: CaCO3
abc   :   4.988000   4.988000  17.061000
angles:  90.000000  90.000000 120.000000
pbc   :       True       True       True


### Primitive cell (not used)

In [5]:
# load calcite crystal structure
calcite_prim = Structure.from_file(calcite_file, primitive=True)
calcite_prim.add_oxidation_state_by_element({"Ca": 2, "C": 4, "O": -2})

# print surface slab data
input = calcite_prim
print("Primitive calcite cell")
print(f"Calcite space group is {sga.get_space_group_symbol()}")
print(f"Is calcite lattice hexagonal? {input.lattice.is_hexagonal()}")
print()

info = str(input).split("\n")
for i in range(5):
    print(info[i])

Primitive calcite cell
Calcite space group is R-3c
Is calcite lattice hexagonal? False

Full Formula (Ca2 C2 O6)
Reduced Formula: CaCO3
abc   :   4.988000   4.988000   6.374586
angles:  66.968255  66.968255  60.000000
pbc   :       True       True       True


## Crystal surface

In [6]:
# make calcite (1 0 -1 4) surface
calcite_104 = SlabGenerator(
                        # initial structure
                        calcite_conv,
                        # miller index
                        miller_index=miller,
                        # Minimum size in angstroms of layers containing atoms
                        min_slab_size=slab_thickness,
                        # Minimum size in angstroms of layers containing vacuum
                        min_vacuum_size=vacuum_thickness,
                        # LLL reduction of lattice
                        lll_reduce=False,
                        # center the slab in the cell with equal vacuum 
                        # spacing from the top and bottom
                        center_slab=True,
                        # set min_slab_size and min_vac_size in units of 
                        # hkl planes (True) or Angstrom (False/default)
                        in_unit_planes=False, 
                        # reduce any generated slabs to a primitive cell
                        primitive=True,
                        # reorients the lattice parameters such that the 
                        # c direction is along the z axis
                        reorient_lattice=True,
                )

slabs = calcite_104.get_slabs()

Slabs also have a number of unique properties that are important when simulating them. Two very important properties are whether they are symmetric and polar. Polar surfaces can be more difficult to relax and compute because they naturally have a redistribution of charge. There are tricks that can be played in many DFT codes to fix this, but all cause other problems. Non-symmetric slabs make computing surface energies more difficult, as you can only compute the average surface energy of the two surfaces together. Let's run a loop over our surfaces and see which ones are polar and which ones are symmetric.

In [7]:
# print calcite surface information
print(f"There are {len(slabs)} surface structure choices.\n")

for n, slab in enumerate(slabs):
    print(f"Slab {n}\n\tPolar: {slab.is_polar()}\n\tSymmetric: {slab.is_symmetric()}")

There are 1 surface structure choices.

Slab 0
	Polar: True
	Symmetric: False


### Save slabs

In [8]:
n_lines = 6
fname_base = f"{crystal}-{''.join([str(s) for s in miller])}_surface"

In [9]:
for slab in slabs:
    # if slab is not symmetric, skip
    if not slab.is_symmetric():
        warnings.warn(f"Slab is not symmetric.")

    # convert slab to supercell
    supercell = slab * supercell_size

    # get dimensions of supercell
    supercell_dims = supercell.lattice.matrix.diagonal() / 10.0
    supercell_dims = "_".join([f"{dim:.2f}" for dim in supercell_dims])
    fname = f"{fname_base}-{supercell_dims}_nm_size-{slab.is_polar()}_polar-{slab.is_symmetric()}_symmetric"

    # write cif file
    supercell.to(filename=f"{fname}.cif", fmt="cif")
    
    # print supercell info
    print(f"Name: {fname}")
    info = str(supercell).split("\n")
    for i in range(n_lines):
        print(info[i])

    # use OpenBabel to read cif file and add bonds
    mol = ob.OBMol()
    obConversion = ob.OBConversion()
    obConversion.SetInAndOutFormats("cif", "pdb")
    obConversion.ReadFile(mol, f"{fname}.cif")
    
    # iterate through atoms and if atom type is Ca, remove bonds to other atoms
    mol.ConnectTheDots()
    bonds = []
    for atom in ob.OBMolAtomIter(mol):
        if atom.GetType() == "Ca":
            for bond in ob.OBAtomBondIter(atom):
                bonds.append(bond)
    for bond in bonds:
        mol.DeleteBond(bond)

    # print molecule info
    print(f"Processed info")
    print(f"\tNumber of atoms: {mol.NumAtoms()}")
    print(f"\tNumber of bonds: {mol.NumBonds()}")
    print(f"\tNumber of residues: {mol.NumResidues()}")
    # get number of atoms of each type
    atom_types = [atom.GetType() for atom in ob.OBMolAtomIter(mol)]
    atom_types, atom_counts = np.unique(atom_types, return_counts=True)
    print(f"\tAtom counts: {dict(zip(atom_types, atom_counts))}")
    # write pdb file
    obConversion.WriteFile(mol, f"{fname}.pdb")
    # delete cif file
    os.remove(f"{fname}.cif")

    # load broken structure with MDA
    u = mda.Universe(f"{fname}.pdb", guess_bonds=True, vdwradii={"Ca": 0.0, "C": 0.7, "O": 0.6}, topology_format="PDB")
    # remove pdf file
    os.remove(f"{fname}.pdb")

    try:
        # find isolated oxygen atoms and add bonds to the closest carbon atom
        for i, atom in enumerate(u.atoms):
            name = atom.name
            res = atom.residue.resname

            bonds = []
            if res != "UNL":
                if len(atom.bonds) == 0:
                    # find closest carbon atom and add a bond
                    sel_c = u.select_atoms(f"element C and around 1.3 index {i}")
                    n_atom_sel = sel_c.n_atoms
                    assert n_atom_sel == 1, f"{n_atom_sel} carbon atoms found around atom {i}"
                    # add bond between atom and sel_c
                    u.add_bonds([(atom.index, sel_c[0].index)])

            # set atom name
            if name == "CA":
                atom.name = "CA"
            elif name == "C":
                atom.name = "CX1"
            elif name == "O":
                atom.name = "OX1"

        # set residue name and update atom names for oxygens
        for i, atom in enumerate(u.atoms):

            # calcium is a monatomic ion
            if atom.name == "CA":
                atom.residue.resname = "CA"
                continue

            # set residue name and number for oxygens with carbon
            if atom.name in ["OX1", "OX2", "OX3"]:
                atom.residue.resname = "CRB"
                continue
        
            # change carboxylate oxygen atom names
            for j, at in enumerate(atom.bonded_atoms):
                at.name = f"OX{j+1}"

            # set residue name and number for carbonate ion
            group = atom.bonded_atoms + atom
            assert len(group) == 4, f"Carbonate at atom {i} has {len(group)} bonded atoms"
            for j, at in enumerate(group):
                at.residue.resname = "CRB"

        # check bond orders
        for i, atom in enumerate(u.atoms):
            name = atom.name
            res = atom.residue.resname

            # assert that all oxygen atoms are bound to a carbon atom
            if name in ["OX1", "OX2", "OX3"]:
                assert len(atom.bonds) == 1, f"Atom {i} is not bound to any carbon atom"
            # assert that all carbon atoms are bound to 3 oxygen atoms
            if name == "CX1":
                assert len(atom.bonds) == 3, f"Atom {i} is not bound to 3 oxygen atoms"
            # assert that all calcium atoms are bound to no other atoms
            if name == "CA":
                assert len(atom.bonds) == 0, f"Atom {i} is bound to {len(atom.bonds)} atoms"

        # check that no atoms have the same coordinates
        coords = u.atoms.positions
        assert len(coords) == len(np.unique(coords, axis=0)), "Atoms have the same coordinates"


    except AssertionError as e:
        warnings.warn(f"Assertion error: {e}")
        fname = f"{fname}_broken"

    # unwrap all atoms
    transform = [
            transformations.unwrap(u.atoms),
            transformations.center_in_box(u.atoms),
            transformations.wrap(u.atoms, compound='fragments'),
    ]
    u.trajectory.add_transformations(*transform)

    # write the new structure
    u.atoms.write(f"{fname}.pdb", bonds='conect', remarks="CaCO3 crystal structure generated with OB and MDA", reindex=True)

    print("\n")


/tmp/ipykernel_3116566/2204688066.py:4: UserWarning: Slab is not symmetric.
  warnings.warn(f"Slab is not symmetric.")


Name: calcite_297K-001_surface-11.97_12.10_13.65_nm_size-True_polar-False_symmetric
Full Formula (Ca4032 C4032 O12096)
Reduced Formula: CaCO3
abc   : 119.712000 139.664000 136.488000
angles:  90.000000  90.000000 120.000000
pbc   :       True       True       True
Sites (20160)
Processed info
	Number of atoms: 20160
	Number of bonds: 11982
	Number of residues: 0
	Atom counts: {'C2': 2, 'C3': 1287, 'Ca': 4032, 'Cac': 2743, 'O.co2': 8187, 'O2': 3785, 'O3': 124}


/tmp/ipykernel_3116566/2204688066.py:126: UserWarning: Assertion error: Carbonate 4041 has 3 bonded atoms
  warnings.warn(f"Assertion error: {e}")
/home/aglisman/miniconda3/envs/pymatgen/lib/python3.9/site-packages/MDAnalysis/core/groups.py:440: DeprecationWarning: The 'pbc' kwarg has been deprecated and will be removed in version 3.0., please use 'wrap' instead
  warnings.warn("The 'pbc' kwarg has been deprecated and will be "
/home/aglisman/miniconda3/envs/pymatgen/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:1126: UserWarning: Found missing chainIDs. Corresponding atoms will use value of 'X'
  warnings.warn("Found missing chainIDs."
